recognize document 

on page with figure find text refering to the figure and text under the figure

find in the text surrounding text reffering to figure sentences related to the figure description text